# Estimate diffusion

Creates pentemeric ring molecule formed by five patchy particles, and simulates for some time (even creates vmd output). Then it estimates translational and rotational diffusion coefficients assuming isotropic diffusion. The number of particles can be changed between 2 and 5 to obtain the diffusion coefficients of the different structures.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2
import msmrd2.tools.quaternions as quats
import msmrd2.visualization as msmrdvis
from msmrd2.potentials import patchyParticleAngular2
from msmrd2.integrators import overdampedLangevin as odLangevin

In [ ]:
# Define particle list with between two and five particles to estimate the diffusion
# of dimers/ trimers / cuatrimers? and pentamer
np.random.seed(seed=1) # seed 1 good for pentamer 
numparticles = 5 # It has value between 2 and 5
boxsize = 3
D = 1.0
Drot = 1.0
pyPartlist = [] 

# Position List for pentamer IC
th = 2*np.pi/5.0
thextra = np.pi
for i in range(numparticles):
    position = 0.85 * np.array([np.cos(th*i),np.sin(th*i),0.0])
    orientation = np.array([np.cos(0.5*(thextra - th*i)),0,0,np.sin(0.5*(thextra - th*i))])
    part = msmrd2.particle(D, Drot, position, quats.conjugate(orientation))
    pyPartlist.append(part)

In [ ]:
# Create list of particles that can be read from msmrd
# Note the particles in this list will be independent from the python list.
partlist = msmrd2.integrators.particleList(pyPartlist)

In [ ]:
# Over-damped Langevin integrator definition
dt = 0.00001 #0.000005
seed = -1 #1 #-1 # Negative seed, uses random device as seed
bodytype = 'rigidbody'
integrator = odLangevin(dt, seed, bodytype) 

In [ ]:
# Define boundary (choose either spherical or box)
# sphericalBoundary = msmrd2.sphere(radius,'reflective')
#boxBoundary = msmrd2.box(boxsize,boxsize,boxsize,'reflective')
#integrator.setBoundary(boxBoundary)

In [ ]:
# Define Patchy Particle potential
sigma = 1.0
strength = 160 #100# 60 #200.0
angularStrength = 20 #10 #200.0
angleDiff = 3*np.pi/5.0
patch1 = np.array([np.cos(angleDiff/2),np.sin(angleDiff/2),0.])
patch2 = np.array([np.cos(-angleDiff/2),np.sin(-angleDiff/2),0.])
patchesCoordinates = [patch1, patch2]
if numparticles > 1:
    potentialPatchyParticleAngular2 = patchyParticleAngular2(sigma, strength, angularStrength, patchesCoordinates)
    integrator.setPairPotential(potentialPatchyParticleAngular2)

In [ ]:
# Define arrays to calculate autocorrelation functions
pentamerPositionArray = []
pentamerOrientationArray = []

In [ ]:
#Integrate particle list and print only positions 
timesteps = 100000 #1000000 #1000000 #20000000
stride = 10 #25 #250 #1000
datafile  = open('../../data/vmd/pentamerTest_' + str(numparticles) + 'particles.xyz', 'w')
cross = [None]*2
# Define reference vectors for plotting reference system
refVec4 = np.array([0., 0., 3.])
refVec5 = np.array([0., 0., -3.])
for i in range(timesteps):
    if i%stride == 0:
        datafile.write(str(3*len(partlist) + 3) + '\n')
        datafile.write(str(0) + '\n')
        if numparticles == 1:
            pentamerPositionArray.append(partlist[0].position)
            pentamerOrientationArray.append(partlist[0].orientation)
        else:
            # Calculate and plot center of pentamer position
            relpos = partlist[1].position - partlist[0].position
            relpos = relpos/np.linalg.norm(relpos)
            patch1 = 0.5*sigma*quats.rotateVec(patchesCoordinates[0], partlist[0].orientation)
            patch2 = 0.5*sigma*quats.rotateVec(patchesCoordinates[1], partlist[0].orientation)
            cross[0] = np.cross(relpos,patch1)
            cross[1] = np.cross(relpos,patch2)
            maxIndex = np.argmax(np.linalg.norm(cross, axis=1))
            rotAxis = cross[maxIndex]/np.linalg.norm(cross[maxIndex])
            rotation = 3*np.pi*rotAxis/10.0
            quatRotation = quats.angle2quat(rotation)
            pentamerCenter = 0.85*quats.rotateVec(relpos,quatRotation)
            pentamerCenter = pentamerCenter + partlist[0].position
            pentamerPositionArray.append(pentamerCenter)
            # Calculate and plot orientation of pentamer (using only the orientation of particle 0)
            orientation0 = np.array([np.cos(0.5*(thextra)),0,0,np.sin(0.5*(thextra))])
            vec1 = 0.85 * np.array([1.,0.,0.]) + 0.5*sigma*quats.rotateVec(patchesCoordinates[0],orientation0)
            vec2 = 0.85 * np.array([1.,0.,0.]) + 0.5*sigma*quats.rotateVec(patchesCoordinates[1],orientation0)
            rotVec1 = partlist[0].position + 0.5*sigma*quats.rotateVec(patchesCoordinates[0],partlist[0].orientation)
            rotVec2 = partlist[0].position + 0.5*sigma*quats.rotateVec(patchesCoordinates[1],partlist[0].orientation)
            pentamerOrientation = quats.recoverRotationFromVectors(np.array([0.,0.,0.]), vec1, vec2, \
                                        pentamerCenter, rotVec1, rotVec2)
            #pentamerOrientation = quats.recoverRotationFromVectors(np.array([0.,0.,0.]), \
            #                            0.85 * np.array([1.,0.,0.]), \
            #                            0.85 * np.array([np.cos(th),np.sin(th),0.0]), \
            #                            pentamerCenter, partlist[0].position, partlist[1].position)
            pentamerOrientationArray.append(pentamerOrientation)
            # Calculate cross reference
            v0 = pentamerCenter
            v1 = v0 + 0.25*quats.rotateVec(refVec4, pentamerOrientation)
            v2 = v0 + 0.25*quats.rotateVec(refVec5, pentamerOrientation)
            # Plot cross reference
            datafile.write('type_2' + ' ' + ' '.join(map(str, v0)) + '\n')
            datafile.write('type_3' + ' ' + ' '.join(map(str, v1)) + '\n')
            datafile.write('type_3' + ' ' + ' '.join(map(str, v2)) + '\n')
    for j, part in enumerate(partlist):
        if i%stride == 0:
            v0 = part.position
            v1 = v0 + 0.5*sigma*quats.rotateVec(patchesCoordinates[0], part.orientation)
            v2 = v0 + 0.5*sigma*quats.rotateVec(patchesCoordinates[1], part.orientation)
            datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
            datafile.write('type_1' + ' ' + ' '.join(map(str, v1)) + '\n')
            datafile.write('type_1' + ' ' + ' '.join(map(str, v2)) + '\n')
    integrator.integrate(partlist)
    if i%10000 == 0:
        print("Percentage complete: ", 100*i/timesteps, "%", end="\r")
datafile.close()
# Generate TCL script to visualize with VMD
msmrdvis.generateTCL_pentamerTest(numparticles = numparticles, 
                                    outfname = 'pentamerTest_' + str(numparticles) + 'particles', 
                                    tclfname = '../../data/vmd/pentamerTest_' + 
                                    str(numparticles) + 'particles_2vmd.tcl')
print("Percentage complete: ", 100, " %")

In [ ]:
# Calculate auto-correlation/mean-square displacement for several lagtimes
tsteps = len(pentamerPositionArray)
lagtimesIndexes = np.arange(100,2000,100) #np.arange(1,20,1)
lagtimes = np.zeros(len(lagtimesIndexes) + 1)
MSD_3D = np.zeros(len(lagtimesIndexes) + 1)
MSD_3D_orientation = np.zeros(len(lagtimesIndexes) + 1)
for i, lagtimeIndex in enumerate(lagtimesIndexes):
    lagtimes[i+1] = dt * lagtimeIndex * stride
    MSD = 0.0
    MSD_orientation = 0.0
    for j in range(tsteps-lagtimeIndex):
        dr = pentamerPositionArray[j+lagtimeIndex] - pentamerPositionArray[j]
        dq = pentamerOrientationArray[j+lagtimeIndex] - pentamerOrientationArray[j]
        MSD += dr*dr
        MSD_orientation += dq*dq
    MSD = MSD/(tsteps - lagtimeIndex + 1)
    MSD_3D[i+1] = sum(MSD) # D = sum(MSD)/(6*lagtime)
    MSD_orientation = MSD_orientation/(tsteps - lagtimeIndex + 1)
    MSD_3D_orientation[i+1] = sum(MSD_orientation[1:])

In [ ]:
# Least square approximation with numpy
A = np.vstack([lagtimes, np.ones(len(lagtimes))]).T
slope,b = np.linalg.lstsq(A, MSD_3D/6, rcond=None)[0]
print(slope,b)
# Least square approximation with numpy (rotation)
y = - np.log(1-4*MSD_3D_orientation/3.0)/2
slope2,b2 = np.linalg.lstsq(A, y, rcond=None)[0]
print(slope2,b2)
# Least square approximation version 2 with numpy (rotation)
y = MSD_3D_orientation*2.0/3.0
slope3,b3 = np.linalg.lstsq(A, y, rcond=None)[0]
print(slope3,b3)

In [ ]:
# Plot MSD against lagtime
plt.plot(lagtimes, MSD_3D/6, 'o', label = 'position')
plt.plot(lagtimes, - np.log(1-4*MSD_3D_orientation/3.0)/2, 'o', label='orientation')
plt.plot(lagtimes, slope*lagtimes + b, '-', label = 'position fit')
plt.plot(lagtimes, slope2*lagtimes + b2, '-', label = 'orientation fit')
#plt.plot(lagtimesRot, 2*MSD_3D_orientation/3, '-o', label='orientation approx')
plt.legend()

In [ ]:
Dapprox = slope
DrotApprox = slope2
print(Dapprox, DrotApprox)